# Decision Generator (Anchored, Recency-Aware, TAST-Driven, Risk-Capped)

This notebook generates the next-round decision files:
- `data/prices/prices_<vNext>.json`
- `data/amounts/amounts_<vNext>.json`
- `data/schedules/schedules_<vNext>.json`

It reads `model_data.json` (from the Model & Analysis Generator) and applies:
- **Anchor pricing from the best-selling week** (per product)
- **TAST_norm** (time-adjusted sell-through) for demand intensity
- **Recency decay** (penalize items with long time since last sale)
- **Dead-item suppression** (stronger reductions for zero recent sales)
- **Risk-capped amounts** (avoid expensive overstock)
- **Schedule by ROI** (revenue / staff cost, once per version)

Exports an **audit CSV** and prints quick anomaly checks.

In [1]:
# === Setup & Parameters ======================================================
import json, math
from pathlib import Path
import numpy as np
import pandas as pd

# Paths
DATA_DIR = Path("data")
SCHEDULES_DIR = DATA_DIR / "schedules"
OUT_DIR = Path("output"); OUT_DIR.mkdir(exist_ok=True, parents=True)

MODEL_DATA_PATH = Path("model_data.json")  # produced by model_and_analysis_generator

# ==== ENDGAME PARAMETER TUNING (salvage-aware, volume-first) ==================
# Prices: slightly down for strong demand (to push volume),
# clearly down for weak/"dead" items (clearance)

PRICE_UP_HOT      = -0.03  # -1% for very hot demand (less discount than before, protect margin)
PRICE_UP_GOOD     = -0.05  # -3% for good demand
PRICE_DOWN_WEAK   = 0.20   # -15% for weak demand (force sell-through)
PRICE_DOWN_DEAD   = 0.35   # -35% for "dead" items (clear out to avoid costly leftover stock)

# Minimum margin & per-round caps (rounding/step limits)
MIN_MARGIN_RATE   = 0.06   # slightly lower to enable volume without going negative
ROUND_CAP_STD     = 0.15   # ±15% allowed per round (more aggressive than before)
ROUND_CAP_DEAD    = 0.30   # ±30% allowed for "dead" items

# Quantities: higher base, strong boost for hot; heavy cut for dead
AMT_BASE_BUFFER   = 1.50   # +80% of max observed demand
AMT_BOOST_HOT     = 2.00   # +100% on top for very hot products
AMT_DEAD_FACTOR   = 0.15   # keep only 15% for "dead"
RISK_CAP_MIN      = 1.40   # increase minimum inventory variance
RISK_CAP_MAX      = 2.40   # allow up to 240% of average (endgame, cheap push)

# Recency / smoothing
EMA_ALPHA         = 0.95   # very strong emphasis on last round(s)
RECENCY_DECAY     = 0.20   # faster devaluation of older data
RECENCY_MIN       = 0.45   # allow sharp downward correction

# TAST / sell-through thresholds
TAST_HOT          = 0.90   # 90%+ considered "hot"
TAST_GOOD_LOW     = 0.80   # from 50% already "good"
TAST_WEAK_HIGH    = 0.60   # <30% is "weak" → strong cuts

# Optional: product name filters for quick audits
ANOMALY_FILTERS   = ["dinosaur", "mattress", "laderhosen", "gjokur_ja"]

In [2]:
# === Load model data + helpers ==============================================
# Load the processed model data generated previously and prepare helper functions.

assert MODEL_DATA_PATH.exists(), "model_data.json not found. Run model_and_analysis_generator first."
md = pd.DataFrame(json.load(open(MODEL_DATA_PATH, "r", encoding="utf-8")))

# Determine version indices
VERSIONS = sorted(md["version"].unique())
V_NOW  = VERSIONS[-1]           # most recent version
HIST_VERS = [v for v in VERSIONS if v < V_NOW]  # historical versions to learn from
V_NEXT = V_NOW + 1              # next version to generate

print("Versions:", VERSIONS, "| Learn from:", HIST_VERS, "| Next:", V_NEXT)


def safe_div(a, b):
    """Safe division: returns 0 where denominator is zero."""
    a = np.asarray(a, float)
    b = np.asarray(b, float)
    return np.divide(a, b, out=np.zeros_like(a), where=b != 0)


def ema_series(x, alpha=EMA_ALPHA):
    """Compute Exponential Moving Average manually over a sequence."""
    s = None
    for xi in x:
        s = alpha * xi + (1 - alpha) * (s if s is not None else xi)
    return s if s is not None else 0.0

Versions: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)] | Learn from: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)] | Next: 7


In [3]:
# === Build historical table & select anchors =================================
# Aggregate the essential metrics per (version, product)

g = (md.groupby(["version", "product"], as_index=False)
        .agg(qty=("qty", "sum"),                # total sold quantity
             amount=("amount", "mean"),         # stocked amount (mean in case multiple rows exist)
             price=("price", "mean"),           # selling price
             revenue=("revenue", "sum"),        # total revenue
             supplier_price=("supplier_price", "mean"),  # cost from supplier
             TAST_norm=("TAST_norm", "mean")))  # time-adjusted sell-through (already computed)

# Keep only historical versions (used to learn from past performance)
g_hist = g[g["version"].isin(HIST_VERS)].copy()

# Select anchor entries:
# For each product: choose the version that had highest sales performance.
# Sorting criteria (priority order):
#   1. Highest qty sold
#   2. Higher revenue
#   3. Lower price (meaning still sold even though price was low)
g_sorted = g_hist.sort_values(
    ["product", "qty", "revenue", "price"],
    ascending=[True, False, False, True]
)

# Take the "best" entry (first after sorting rules)
anchor = (
    g_sorted.groupby("product", as_index=False)
            .first()[["product", "version", "price", "qty", "revenue", "amount", "supplier_price"]]
            .rename(columns={
                "version": "anchor_v",
                "price": "p_anchor",
                "qty": "anchor_qty",
                "revenue": "anchor_revenue",
                "amount": "anchor_amount",
                "supplier_price": "supplier_price_ref"
            })
)

print("Anchor sample:")
display(anchor.head(8))

Anchor sample:


,product,anchor_v,p_anchor,anchor_qty,anchor_revenue,anchor_amount,supplier_price_ref
0,batteries,4,17.65,3285.0,57980.25,3285.0,12.0
1,dinosaur,1,94.29,448.0,42241.92,1678.0,65.0
2,gjokur_ja,5,893.63,100.0,89363.00,100.0,580.0
3,hammer,4,63.43,4007.0,254164.01,4007.0,45.0
4,hot_dogs,5,27.04,3004.0,81228.16,3004.0,18.0
5,ice_cream,4,12.09,3437.0,41553.33,3437.0,8.0
6,knives,5,49.88,1628.0,81204.64,1628.0,38.0
7,laderhosen,5,301.10,414.0,124655.40,414.0,220.0


In [4]:
# === Indicators: TAST-EMA, VOL, Risk, Recency ================================
# Build per-product indicators from historical data to guide next-round pricing and stocking.

# Compute historical metrics per product
hist = (
    g_hist.groupby("product")
          .apply(lambda d: pd.Series({
              # EMA of time-adjusted sell-through (demand signal)
              "TAST_EMA": ema_series(d["TAST_norm"].fillna(0.0), alpha=EMA_ALPHA),

              # Max quantity ever sold in a version (upper volume potential)
              "qty_max": d["qty"].max(),

              # Average amount stocked historically
              "amount_avg": d["amount"].mean(),

              # Sales volatility (standard deviation of qty)
              "VOL": float(d["qty"].std(ddof=0)) if len(d) > 1 else 0.0
          }))
          .reset_index()
)

# Median price and cost per product (used for risk scaling)
p_med = g_hist.groupby("product")["price"].median().rename("price_med")
c_med = g_hist.groupby("product")["supplier_price"].median().rename("cost_med")

# Merge anchors with calculated historical indicators
ref = (
    anchor.merge(hist, on="product", how="left")
          .merge(p_med, on="product", how="left")
          .merge(c_med, on="product", how="left")
)

# Price-to-cost ratios (absolute vs median reference)
ref["PCR"]     = safe_div(ref["p_anchor"], ref["supplier_price_ref"])
ref["PCR_med"] = safe_div(ref["price_med"], ref["cost_med"])

# Risk score: high score → reduce aggressive stock increases
# Weights: price-to-cost ratio dominance, price comparison, demand volatility
w1, w2, w3 = 0.5, 0.3, 0.2
VOL_med = ref["VOL"].median() if ref["VOL"].median() > 0 else 1.0

ref["RISK"] = (
    (ref["PCR"] / ref["PCR_med"]).replace([np.inf, -np.inf], np.nan).fillna(1.0) * w1 +
    (ref["p_anchor"] / ref["price_med"]).replace([np.inf, -np.inf], np.nan).fillna(1.0) * w2 +
    (ref["VOL"] / VOL_med).replace([np.inf, -np.inf], np.nan).fillna(1.0) * w3
)

# Dead items: no sales in last 2 historical rounds
last_weeks = sorted(HIST_VERS)[-2:] if len(HIST_VERS) >= 2 else HIST_VERS
recent = g_hist[g_hist["version"].isin(last_weeks)].groupby("product")["qty"].sum()
ref = ref.merge(recent.rename("qty_recent"), on="product", how="left").fillna({"qty_recent": 0.0})
ref["is_dead"] = ref["qty_recent"] <= 0

# Recency: number of versions since last sale
last_sale_v = (g[g["qty"] > 0].groupby("product")["version"].max().rename("last_sold_version"))
ref = ref.merge(last_sale_v, on="product", how="left")
ref["weeks_since_sale"] = V_NOW - ref["last_sold_version"].fillna(0)

# Apply decay multiplier to scale down unproven or old-performing products
ref["recency_multiplier"] = (
    1.0 - RECENCY_DECAY * ref["weeks_since_sale"]
).clip(lower=RECENCY_MIN, upper=1.0)

print("Indicators sample:")
display(ref.head(8))

Indicators sample:


/tmp/ipykernel_21285/2855663262.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({


,product,anchor_v,p_anchor,anchor_qty,anchor_revenue,anchor_amount,supplier_price_ref,TAST_EMA,qty_max,amount_avg,...,price_med,cost_med,PCR,PCR_med,RISK,qty_recent,is_dead,last_sold_version,weeks_since_sale,recency_multiplier
0,batteries,4,17.65,3285.0,57980.25,3285.0,12.0,1.000000,3285.0,2504.333333,...,16.330,12.0,1.470833,1.360833,1.064666,6501.0,False,6,0,1.0
1,dinosaur,1,94.29,448.0,42241.92,1678.0,65.0,0.496739,448.0,1669.166667,...,91.440,65.0,1.450615,1.406769,0.880055,253.0,False,6,0,1.0
2,gjokur_ja,5,893.63,100.0,89363.00,100.0,580.0,0.962229,100.0,76.500000,...,859.145,580.0,1.540741,1.481284,0.840184,117.0,False,6,0,1.0
3,hammer,4,63.43,4007.0,254164.01,4007.0,45.0,0.874041,4007.0,3105.500000,...,64.400,45.0,1.409556,1.431111,0.989229,7422.0,False,6,0,1.0
4,hot_dogs,5,27.04,3004.0,81228.16,3004.0,18.0,0.999999,3004.0,2264.666667,...,25.975,18.0,1.502222,1.443056,1.006029,5692.0,False,6,0,1.0
5,ice_cream,4,12.09,3437.0,41553.33,3437.0,8.0,1.000000,3437.0,2649.000000,...,11.255,8.0,1.511250,1.406875,1.095179,6831.0,False,6,0,1.0
6,knives,5,49.88,1628.0,81204.64,1628.0,38.0,0.997970,1628.0,2451.166667,...,55.205,38.0,1.312632,1.452763,0.867082,2984.0,False,6,0,1.0
7,laderhosen,5,301.10,414.0,124655.40,414.0,220.0,0.997604,414.0,1809.333333,...,309.825,220.0,1.368636,1.408295,0.812050,759.0,False,6,0,1.0


In [5]:
# === Price rule (anchored + TAST / recency / dead guards) ====================
def price_next(r):
    """
    Compute next-round price based on:
      - Anchored price from historically best round (p_anchor)
      - Demand strength via TAST_EMA (time-adjusted sell-through)
      - Dead-product logic (no recent sales → strong markdown)
      - Recency decay (older products get progressively cheaper)
      - Margin and safety caps (per-round max adjustment allowed)
    """

    p0   = float(r["p_anchor"] or 0.0)              # historic anchor price
    sp   = float(r["supplier_price_ref"] or 0.0)    # supplier cost reference
    tast = float(r["TAST_EMA"] or 0.0)              # demand strength (EMA of TAST)
    dead = bool(r["is_dead"])                       # “dead”: zero sales recently
    rec  = float(r["recency_multiplier"] or 1.0)    # recency multiplier (0.45–1.0)

    # Start from anchor price
    p = p0

    # Demand-dependent pricing logic
    if dead:
        # Dead item → heavy discount (clearance)
        p = p0 * (1.0 - PRICE_DOWN_DEAD)
    elif tast >= TAST_HOT:
        # Very strong demand → slight discount to boost volume, protect margin
        p = p0 * (1.0 + PRICE_UP_HOT)
    elif TAST_GOOD_LOW <= tast < TAST_HOT:
        # Good demand → slightly stronger discount to push more volume
        p = p0 * (1.0 + PRICE_UP_GOOD)
    elif tast <= TAST_WEAK_HIGH and (r["amount_avg"] or 0) > 0:
        # Weak demand → larger discount
        p = p0 * (1.0 - PRICE_DOWN_WEAK)

    # Apply recency penalty (stale products get cheaper)
    p *= rec

    # Enforce minimum margin (avoid selling below supplier cost + min margin)
    min_allowed = sp * (1.0 + MIN_MARGIN_RATE) if sp > 0 else 0.0
    p = max(p, min_allowed)

    # Limit aggressive price changes per iteration
    cap_up   = p0 * (1.0 + (ROUND_CAP_DEAD if dead else ROUND_CAP_STD))
    cap_down = p0 * (1.0 - (ROUND_CAP_DEAD if dead else ROUND_CAP_STD))
    p = min(max(p, cap_down), cap_up)

    return round(float(p), 2)

In [6]:
# === Amount rule (TAST + risk cap + recency/dead) ============================
def amount_next(r):
    """
    Determine next-round stock amount based on:
      - Demand strength (TAST_EMA)
      - Historical max quantity sold (qty_max)
      - Historical average stock level (amount_avg)
      - Risk score (influences how aggressive we allow stock increases)
      - Recency decay (products not selling recently get reduced amounts)
      - Dead-product logic (drastic downscale)
    """

    tast    = float(r["TAST_EMA"] or 0.0)          # demand strength (time-adjusted EMA)
    qty_max = float(r["qty_max"] or 0.0)           # highest historical quantity sold
    amt_avg = float(r["amount_avg"] or 0.0)        # historical average stocked amount
    risk    = float(r["RISK"] or 1.0)              # risk score (higher → more conservative)
    dead    = bool(r["is_dead"])                   # flag: zero recent sales → treat as dead item
    rec     = float(r["recency_multiplier"] or 1.0)# recency downscale factor (≤1 for stale items)

    # Base amount = > observed demand (scaled buffer)
    base = math.ceil(qty_max * AMT_BASE_BUFFER)

    # If extremely hot demand → boost stocking aggressively
    if tast > 0.95:
        base = math.ceil(base * AMT_BOOST_HOT)

    # Dead item → stock only a minimal fraction
    if dead:
        base = max(0, round(amt_avg * AMT_DEAD_FACTOR))

    # Risk-cap: convert risk score into allowed inventory upper bound
    #   Higher risk → smaller allowed multiplier
    cap_mult = np.clip(1.0 + 0.5 / (risk if risk > 0 else 1.0),
                       RISK_CAP_MIN, RISK_CAP_MAX)

    # Maximum allowed amount based on risk scaling
    cap = amt_avg * cap_mult
    if cap > 0:
        base = min(base, math.ceil(cap))

    # Recency downscale (lower for products that haven't sold recently)
    base = int(base * rec)

    return int(max(0, base))

In [7]:
# === Schedule selection by ROI ==============================================
# Select which past schedule to reuse based on ROI (Revenue / Staff cost)

# Compute total revenue per version
rev_by_v = md.groupby("version")["revenue"].sum()

# Staff cost is version-level, so take only one entry per version (max or first, both valid)
staff_by_v = md.groupby("version")["version_staff_cost"].max()

# ROI = revenue earned per unit staff cost
roi = (rev_by_v / staff_by_v.replace(0, np.nan)).dropna()

# Consider only historical versions (learning set)
roi_hist = roi[roi.index.isin(HIST_VERS)]

if not roi_hist.empty:
    # Select version with highest ROI
    sched_src_v = int(roi_hist.idxmax())
else:
    # Fallback:
    # If ROI cannot be calculated, choose most recent historical version
    # that actually has a schedule JSON file
    candidates = [v for v in reversed(HIST_VERS)
                  if (SCHEDULES_DIR / f"schedules_{v}.json").exists()]
    sched_src_v = candidates[0] if candidates else HIST_VERS[-1]

# Load selected schedule to use as template for the next version
with open(SCHEDULES_DIR / f"schedules_{sched_src_v}.json", "r", encoding="utf-8") as f:
    schedules_next = json.load(f)

print(f"Schedule source version (best ROI among history): v{sched_src_v}")

Schedule source version (best ROI among history): v5


In [8]:
# === Generate JSONs & Audit ==================================================
# Produce next-round decisions:
#   - prices_{V_NEXT}.json
#   - amounts_{V_NEXT}.json
#   - schedules_{V_NEXT}.json
# Also create an audit CSV summarizing decisions per product.

# Apply rules to generate next prices and amounts
prices_next  = {r["product"]: price_next(r)  for _, r in ref.iterrows()}
amounts_next = {r["product"]: amount_next(r) for _, r in ref.iterrows()}

# Ensure directories exist before writing output files
(DATA_DIR / "prices").mkdir(parents=True, exist_ok=True)
(DATA_DIR / "amounts").mkdir(parents=True, exist_ok=True)
(DATA_DIR / "schedules").mkdir(parents=True, exist_ok=True)

# Write decision JSONs
with open(DATA_DIR / "prices" / f"prices_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(prices_next, f, indent=2, ensure_ascii=False)

with open(DATA_DIR / "amounts" / f"amounts_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(amounts_next, f, indent=2, ensure_ascii=False)

with open(DATA_DIR / "schedules" / f"schedules_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(schedules_next, f, indent=2, ensure_ascii=False)

# === Audit table =============================================================
# Display key indicators + pricing/amount decisions for transparency

audit = ref[[
    "product", "anchor_v", "p_anchor", "qty_max", "amount_avg",
    "TAST_EMA", "RISK", "is_dead", "weeks_since_sale", "recency_multiplier"
]].copy()

audit["price_next"]  = audit["product"].map(prices_next)
audit["amount_next"] = audit["product"].map(amounts_next)

# Price delta relative to anchor (percent)
audit["d_price_vs_anchor_%"] = (
    (audit["price_next"] - audit["p_anchor"]) / audit["p_anchor"] * 100
)

# Save audit for debugging/traceability
audit_path = OUT_DIR / f"decision_v{V_NEXT}_audit.csv"
audit.to_csv(audit_path, index=False)

print("Generated decisions:")
print(" -", DATA_DIR/"prices"/f"prices_{V_NEXT}.json")
print(" -", DATA_DIR/"amounts"/f"amounts_{V_NEXT}.json")
print(" -", DATA_DIR/"schedules"/f"schedules_{V_NEXT}.json")
print("Audit:", audit_path)

# Show most relevant items first in audit display
display(
    audit.sort_values(
        ["is_dead", "weeks_since_sale", "RISK"],
        ascending=[False, False, False]
    ).head(15)
)

Generated decisions:
 - data/prices/prices_7.json
 - data/amounts/amounts_7.json
 - data/schedules/schedules_7.json
Audit: output/decision_v7_audit.csv


,product,anchor_v,p_anchor,qty_max,amount_avg,TAST_EMA,RISK,is_dead,weeks_since_sale,recency_multiplier,price_next,amount_next,d_price_vs_anchor_%
11,rice_porridge,4,23.26,3911.0,2974.833333,1.000000,1.120096,False,0,1.0,22.56,4303,-3.009458
5,ice_cream,4,12.09,3437.0,2649.000000,1.000000,1.095179,False,0,1.0,11.73,3859,-2.977667
0,batteries,4,17.65,3285.0,2504.333333,1.000000,1.064666,False,0,1.0,17.12,3681,-3.002833
9,monster,4,21.99,3111.0,2547.666667,0.999997,1.061997,False,0,1.0,21.33,3748,-3.001364
10,nails,5,8.64,4152.0,3254.000000,1.000000,1.055776,False,0,1.0,8.38,4796,-3.009259
12,sunscreen,5,53.85,2999.0,2339.666667,1.000000,1.030292,False,0,1.0,52.23,3476,-3.008357
4,hot_dogs,5,27.04,3004.0,2264.666667,0.999999,1.006029,False,0,1.0,26.23,3391,-2.995562
8,mattress,0,372.34,2161.0,2664.000000,0.460529,1.001033,False,0,1.0,316.49,3242,-14.999731
3,hammer,4,63.43,4007.0,3105.500000,0.874041,0.989229,False,0,1.0,60.26,4676,-4.997635
1,dinosaur,1,94.29,448.0,1669.166667,0.496739,0.880055,False,0,1.0,80.15,672,-14.996288


In [9]:
# === Quick anomaly checks ====================================================
def show_anomalies(filters=ANOMALY_FILTERS, df=audit):
    """
    Display products whose names match any token in ANOMALY_FILTERS.
    Purpose:
      - Quickly inspect suspicious / extreme items (e.g., weird product names)
      - Helps verify that price/amount decisions are reasonable for outliers
    """

    if not filters:
        print("No filters provided.")
        return

    # Build OR-filter over all tokens (case-insensitive product name match)
    flt = None
    for token in filters:
        # True for matching products
        m = df["product"].str.contains(token, case=False, na=False)
        flt = m if flt is None else (flt | m)

    # Filter table based on accumulated mask
    res = df[flt] if flt is not None else pd.DataFrame()

    # Output results
    if res.empty:
        print("No products matched filters:", filters)
    else:
        print("Anomaly candidates:")
        display(
            res.sort_values(
                ["is_dead", "weeks_since_sale", "RISK"],
                ascending=[False, False, False]
            )
        )

# Run anomaly check using predefined filters
show_anomalies()

Anomaly candidates:


,product,anchor_v,p_anchor,qty_max,amount_avg,TAST_EMA,RISK,is_dead,weeks_since_sale,recency_multiplier,price_next,amount_next,d_price_vs_anchor_%
8,mattress,0,372.34,2161.0,2664.000000,0.460529,1.001033,False,0,1.0,316.49,3242,-14.999731
1,dinosaur,1,94.29,448.0,1669.166667,0.496739,0.880055,False,0,1.0,80.15,672,-14.996288
2,gjokur_ja,5,893.63,100.0,76.500000,0.962229,0.840184,False,0,1.0,866.82,123,-3.000123
7,laderhosen,5,301.10,414.0,1809.333333,0.997604,0.812050,False,0,1.0,292.07,1242,-2.999004


### Notes & Rationale

- **TAST_norm** (time-adjusted sell-through) replaces raw sell-through as primary demand signal.
- **Anchor pricing** avoids using stale or depressed last-week prices; we anchor to the best-selling week.
- **Recency decay** reduces both price and amount for items with a long time since last sale.
- **Dead-item suppression** applies stronger reductions and lower restock targets.
- **Risk cap** prevents big stock increases on expensive or volatile items.
- **Schedule ROI** picks the historical week with the best revenue/staff-cost ratio.

Per-round caps keep changes conservative in a competitive environment.
If any product looks odd in the audit, tune:
- `RECENCY_DECAY` (0.05–0.15), `RECENCY_MIN` (0.5–0.8),
- amount caps (`RISK_CAP_MIN/MAX`),
- price caps (`ROUND_CAP_STD/DEAD`),
- TAST thresholds (`TAST_HOT`, `TAST_GOOD_LOW`, `TAST_WEAK_HIGH`).